In [1]:
# Quick Folium map: Manhattan rates + bus lanes + schools/hospitals
import os, json, ast
from pathlib import Path
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import folium
from folium import plugins

DATA = Path.cwd().parent / "data"
RAW = DATA / "raw"
PROCESSED = DATA / "processed"
OUT_DIR = Path.cwd().parent / "reports" / "dashboards"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 1) Load rates and NTA polygons
rates = pd.read_csv(PROCESSED / "rates_by_nta_hour_manhattan.csv")
rates_mean = rates.groupby(["nta2020"], as_index=False)["rate"].mean()

import requests
r = requests.get(
    "https://data.cityofnewyork.us/resource/9nt8-h7nd.json",
    params={"$where": "borocode=1", "$limit": 50000}, timeout=60
)
r.raise_for_status()
ntas_df = pd.DataFrame(r.json())
ntas_gdf = gpd.GeoDataFrame(
    ntas_df,
    geometry=[shape(g) if isinstance(g, dict) else None for g in ntas_df.get("the_geom", [])],
    crs="EPSG:4326"
)
ntas_gdf = ntas_gdf.merge(rates_mean, on="nta2020", how="left")
# Keep only NTAs with lane miles (non-null rate) for the heat layer
ntas_with_rate = ntas_gdf[ntas_gdf["rate"].notna()].copy()

# 2) Prepare bus lanes GeoJSON features
lanes_df = pd.read_csv(RAW / "bus_lanes_manhattan.csv")
features_lanes = []
for _, row in lanes_df.iterrows():
    try:
        geom = row.get("the_geom")
        if isinstance(geom, str):
            geom = ast.literal_eval(geom)
        if isinstance(geom, dict):
            features_lanes.append({
                "type": "Feature",
                "geometry": geom,
                "properties": {"street": row.get("street", ""), "lane_type1": row.get("lane_type1", "")}
            })
    except Exception:
        pass
lanes_geojson = {"type": "FeatureCollection", "features": features_lanes[:2000]}  # cap for speed

# 3) Parse schools and hospitals points (Manhattan files)
def load_points_csv(path, field="location_1"):
    pts = []
    if not path.exists():
        return pts
    df = pd.read_csv(path)
    # Case A: latitude/longitude numeric columns exist
    for lat_col, lon_col in [("latitude","longitude"),("lat","lon"),("LATITUDE","LONGITUDE")]:
        if lat_col in df.columns and lon_col in df.columns:
            sdf = df[[lat_col, lon_col]].apply(pd.to_numeric, errors="coerce").dropna()
            for idx, row_vals in sdf.iterrows():
                lat, lon = float(row_vals[lat_col]), float(row_vals[lon_col])
                pts.append((lat, lon, df.loc[idx]))
            return pts
    # Case B: Socrata point dict in string
    for _, row in df.iterrows():
        loc = row.get(field)
        if isinstance(loc, str):
            try:
                loc = ast.literal_eval(loc)
            except Exception:
                loc = None
        if isinstance(loc, dict):
            coords = loc.get("coordinates")
            if coords and len(coords) == 2:
                pts.append((coords[1], coords[0], row))
    return pts

schools_pts = load_points_csv(RAW / "schools_manhattan.csv")
hosp_pts = load_points_csv(RAW / "hospitals_manhattan.csv")

# 4) Build Folium map
m = folium.Map(location=[40.78, -73.97], zoom_start=12, tiles="CartoDB positron")

# Light gray outline for all Manhattan NTAs
folium.GeoJson(
    ntas_gdf[["ntaname","nta2020","geometry"]].to_json(),
    name="NTA outline",
    style_function=lambda f: {"color": "#bbb", "weight": 0.8, "fillOpacity": 0}
).add_to(m)

# Choropleth for mean rate
folium.Choropleth(
    geo_data=ntas_with_rate.to_json(),
    data=ntas_with_rate[["nta2020","rate"]],
    columns=["nta2020", "rate"],
    key_on="feature.properties.nta2020",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_opacity=0.0,
    legend_name="Violation rate (violations per lane-mile)"
).add_to(m)

# Add hover labels for NTA name + rate
folium.GeoJson(
    ntas_with_rate[["ntaname","nta2020","rate","geometry"]].to_json(),
    name="NTA labels",
    style_function=lambda f: {"color": "#999", "weight": 0.3, "fillOpacity": 0},
    tooltip=folium.features.GeoJsonTooltip(
        fields=["ntaname","nta2020","rate"],
        aliases=["NTA","Code","Rate"],
        localize=True,
        sticky=False
    )
).add_to(m)

# Bus lanes layer
folium.GeoJson(
    lanes_geojson,
    name="Bus Lanes",
    style_function=lambda f: {"color": "#c33", "weight": 2, "opacity": 0.7}
).add_to(m)

# Add mini-map and fullscreen controls
plugins.MiniMap(toggle_display=True).add_to(m)
plugins.Fullscreen().add_to(m)

# Schools and hospitals (with hover labels)
schools_grp = folium.FeatureGroup(name="Schools", show=False).add_to(m)
hosp_grp = folium.FeatureGroup(name="Hospitals", show=True).add_to(m)
for lat, lon, row in schools_pts:
    name = None
    if row is not None:
        name = row.get("school_name") or row.get("facility_name") or "School"
    folium.CircleMarker(location=[lat, lon], radius=2, color="#1f77b4", fill=True, fill_opacity=0.8,
                        tooltip=name).add_to(schools_grp)
for lat, lon, row in hosp_pts:
    name = None
    if row is not None:
        name = row.get("facility_name") or row.get("facility_type") or "Hospital"
    folium.CircleMarker(location=[lat, lon], radius=3, color="#d62728", fill=True, fill_opacity=0.9,
                        tooltip=name).add_to(hosp_grp)

# Action pins: highlight top 10 from playbook at NTA centroids
playbook_path = PROCESSED / "where_when_action_playbook_mn.csv"
if playbook_path.exists():
    try:
        pb = pd.read_csv(playbook_path).head(10)
        pb_join = ntas_gdf[["nta2020","ntaname","geometry"]].merge(pb, on=["nta2020","ntaname"], how="inner")
        actions_grp = folium.FeatureGroup(name="Action: Top 10", show=True).add_to(m)
        for _, row in pb_join.iterrows():
            pt = row["geometry"].representative_point()
            lat, lon = pt.y, pt.x
            popup_html = f"<b>{row['ntaname']}</b><br>Hour: {int(row['hour'])}:00<br>Rate: {row['rate']:.2f}<br><i>{row['window']}</i><br>{row['recommendation']}"
            folium.Marker(
                location=[lat, lon],
                popup=popup_html,
                tooltip=f"{row['ntaname']} (hour {int(row['hour'])})",
                icon=folium.Icon(color="purple", icon="flag", prefix="fa")
            ).add_to(actions_grp)
    except Exception as e:
        pass

folium.LayerControl(collapsed=False).add_to(m)

out_path = OUT_DIR / "map_manhattan_quickcheck.html"
m.save(str(out_path))
print(f"Saved map → {out_path}")


Saved map → /Users/mohamedhiba/Fall 2025/datathon/reports/dashboards/map_manhattan_quickcheck.html
